In [21]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
train_root = "/kaggle/input/emotions-dataset-for-nlp/train.txt"
val_root = "/kaggle/input/emotions-dataset-for-nlp/val.txt"
test_root = "/kaggle/input/emotions-dataset-for-nlp/test.txt"


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [22]:
train_df = pd.read_csv(train_root, delimiter=";")
train_df.columns = ["text", "emotion"]
val_df = pd.read_csv(val_root, delimiter=";")
val_df.columns = ["text", "emotion"]
test_df = pd.read_csv(test_root, delimiter=";")
test_df.columns = ["text", "emotion"]

In [23]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

# Fit and transform the class column
train_df['emotion'] = label_encoder.fit_transform(train_df['emotion'])
val_df['emotion'] = label_encoder.fit_transform(val_df['emotion'])
test_df['emotion'] = label_encoder.fit_transform(test_df['emotion'])


In [4]:
train_df.head()

,text,emotion
0,i can go from feeling so hopeless to so damned...,4
1,im grabbing a minute to post i feel greedy wrong,0
2,i am ever feeling nostalgic about the fireplac...,3
3,i am feeling grouchy,0
4,ive been feeling a little burdened lately wasn...,4


In [5]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora/


Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [7]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [8]:
def preprocess_text(text):
    tokenized = word_tokenize(text)
    
    lemmatizer = WordNetLemmatizer()

    stop_words = set(stopwords.words('english'))
    
    lemmatized = [lemmatizer.lemmatize(word.lower()) for word in tokenized if word.lower() not in stop_words]
    
    return " ".join(lemmatized)

In [9]:
train_df["text"] = train_df["text"].apply(preprocess_text)
val_df["text"] = val_df["text"].apply(preprocess_text)
test_df["text"] = test_df["text"].apply(preprocess_text)


In [10]:
train_df.head()

,text,emotion
0,go feeling hopeless damned hopeful around some...,4
1,im grabbing minute post feel greedy wrong,0
2,ever feeling nostalgic fireplace know still pr...,3
3,feeling grouchy,0
4,ive feeling little burdened lately wasnt sure,4


In [11]:
train_df["emotion"].value_counts()

emotion
2    5362
4    4665
0    2159
1    1937
3    1304
5     572
Name: count, dtype: int64

In [24]:
X_train = train_df["text"]
X_val = val_df["text"]
X_test = test_df["text"]

y_train = train_df["emotion"]
y_val = val_df["emotion"]
y_test = test_df["emotion"]


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# Initialize the vectorizer and SVD model
vectorizer = TfidfVectorizer(ngram_range=(1, 1))
#svd = TruncatedSVD(n_components=1000)  # Adjust n_components as needed

# Fit and transform the training data
X_train_vec = vectorizer.fit_transform(X_train)
# X_train_svd = svd.fit_transform(X_train_vec)

# Transform the validation and test data
X_val_vec = vectorizer.transform(X_val)
# X_val_svd = svd.transform(X_val_vec)

X_test_vec = vectorizer.transform(X_test)
# X_test_svd = svd.transform(X_test_vec)

print(f"Original number of features: {X_train_vec.shape[1]}")
# print(f"Reduced number of features: {X_train_svd.shape[1]}")

Original number of features: 13456


In [ ]:
# grid_search = GridSearchCV(model, {
#     "C": [0.8, 0.9, 1.0], 
#     "kernel": ["linear", "poly", "rbf", "sigmoid"],
#     "degree": [3, 4, 5, 6, 7, 8]
# }, verbose=3)

# grid_search.fit(X_train_vec, y_train)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [ ]:
model = SVC(kernel="linear")
model.fit(X_train_vec, y_train)
preds = model.predict(X_test_vec)

score = accuracy_score(preds, y_test)
score

In [ ]:
model = SVC(kernel="linear", class_weight="balanced")
# 2    5362
# 4    4665
# 0    2159
# 1    1937
# 3    1304
# 5     572
model.fit(X_train_vec, y_train)
preds = model.predict(X_test_vec)
score = accuracy_score(preds, y_test)
score

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train_svd, y_train)
preds = model.predict(X_test_svd)
score = accuracy_score(preds, y_test)
score

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model = LogisticRegression(max_iter=1000)  # You can choose other kernels like 'rbf', 'poly', etc.
model.fit(X_train_svd, y_train)
preds = model.predict(X_test_svd)

score = accuracy_score(preds, y_test)
score

In [ ]:
dtrain = xgb.DMatrix(X_train_svd, label=y_train)
dtest = xgb.DMatrix(X_test_svd, label=y_test)

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

#class_weight = [1.23506253, 1.37661332, 0.49729579, 2.04486196, 0.571597, 4.66171329]
classes = [0, 1, 2, 3, 4, 5]
    
class_weights = {l:w for w,l in zip(class_weight, classes)}

xgb_clf = xgb.XGBClassifier(objective='multi:softmax',
                            learning_rate=0.5,
                            num_class=6, 
                            n_estimators=200,
                            early_stopping_rounds=10, 
                            eval_metric=['merror','mlogloss'], 
                            seed=42)
xgb_clf.fit(X_train_vec, 
            y_train,
            #sample_weight=[class_weights[y] for y in y_train],
            verbose=1, # set to 1 to see xgb training round intermediate results
            eval_set=[(X_train_vec, y_train), (X_test_vec, y_test)])

In [ ]:
preds = xgb_clf.predict(X_val_vec)

In [ ]:
accuracy_score(preds, y_val)

In [ ]:
y_pred_prob = bst.predict(dtest)
y_pred = [1 if pred > 0.5 else 0 for pred in y_pred_prob]

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

In [ ]:
# 91.54% TF-idf-ngram range (1, 2)/XGBoost-0.5 lr, 200 estimators.

In [ ]:
max_len = lambda x:len(x)

In [25]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# Assuming texts is a list of preprocessed text strings
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

# Tokenize the texts
sequences = tokenizer.texts_to_sequences(X_train)
word_index = tokenizer.word_index

print(len(sequences))

max_length = train_df['text'].apply(lambda x: len(x.split())).max() + 1

# Pad the sequences
padded_sequences = pad_sequences(sequences, maxlen=max_length)

# Define the model
vocab_size = len(word_index) + 1

padded_sequences[:2]


15999


array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    1,   39,  100,   59,    7,   14,  493,    4,   14,
        3495,  552,   31,   59,   60,  127,  147,   75, 1479,    3,   21,
        1254],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,   16, 3059,    6, 1148,    4,  285,    1,    2,  494,
         437],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
        

In [26]:
max_length

67

In [27]:
sequences = tokenizer.texts_to_sequences(X_test)
word_index = tokenizer.word_index

print(len(sequences))

# Pad the sequences
test_padded_sequences = pad_sequences(sequences, maxlen=max_length)

1999


In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TextLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers=1, bidirectional=False, dropout=0.5):
        super(TextLSTM, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, num_layers=n_layers, 
                            bidirectional=bidirectional, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        embedded = self.dropout(self.embedding(x))
        lstm_out, (hidden, cell) = self.lstm(embedded)
        
        if self.lstm.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim=1))
        else:
            hidden = self.dropout(hidden[-1,:,:])
        
        output = self.fc(hidden)
        return output

In [29]:
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

X_train_tensor = torch.tensor(padded_sequences)
y_train_tensor = torch.tensor(y_train)
# Assuming X_train_tensor and y_train_tensor are your prepared tensors

batch_size = 64

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

X_test_tensor = torch.tensor(test_padded_sequences)
y_test_tensor = torch.tensor(y_test)
# Assuming X_train_tensor and y_train_tensor are your prepared tensors

test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)


# Model hyperparameters
embedding_dim = 100
hidden_dim = 256
output_dim = len(set(y_train))  # Number of classes
n_layers = 2
bidirectional = True
dropout = 0.5

# Instantiate the model, loss function, and optimizer
model = TextLSTM(vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [30]:
def accuracy_score(loader, model):
    correct = 0
    total = 0
    model.eval()
    with torch.no_grad():
        for X_batch, y_batch in loader:
            predictions = model(X_batch)
            _, predicted = torch.max(predictions.data, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()
    accuracy = 100 * correct / total
    return accuracy

In [31]:
# Training loop
epochs = 10

for epoch in range(epochs):
    model.train()
    total_loss = 0
    model.train()
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        
        # Forward pass
        predictions = model(X_batch)
        
        # Calculate loss
        loss = criterion(predictions, y_batch)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    print(f'Epoch {epoch+1}/{epochs},train loss: {avg_loss:.4f}')
    
    accuracy = accuracy_score(test_loader, model)
    print(f'Accuracy: {(accuracy*100):.4f}%')    


Epoch 1/10,train loss: 1.5648
Accuracy: 4352.1761%
Epoch 2/10,train loss: 1.4553
Accuracy: 5817.9090%
Epoch 3/10,train loss: 1.1763
Accuracy: 7688.8444%
Epoch 4/10,train loss: 0.8348
Accuracy: 8499.2496%
Epoch 5/10,train loss: 0.6082
Accuracy: 8939.4697%
Epoch 6/10,train loss: 0.4550
Accuracy: 9159.5798%
Epoch 7/10,train loss: 0.3808
Accuracy: 9154.5773%
Epoch 8/10,train loss: 0.3124
Accuracy: 9244.6223%
Epoch 9/10,train loss: 0.2792
Accuracy: 9269.6348%
Epoch 10/10,train loss: 0.2502
Accuracy: 9249.6248%


In [32]:
accuracy = accuracy_score(val_loader, model)
print(f'Accuracy: {(accuracy*100):.4f}%')  

NameError: name 'val_loader' is not defined

In [ ]:
# 9264.6323% LSTM 2 hidden layers 100 embedding dims, lemmatized text